In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import datetime as dt
import requests
import json
import os
from bs4 import BeautifulSoup
import time
import random

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
def get_stock_data(symbol, period='1y'):
    time.sleep(2)  # Add 2 second delay for every call
    try:
        ticker = f"{symbol}.NS"
        data = yf.download(ticker, period=period)
        return data
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return pd.DataFrame()

In [ ]:
def calculate_rsi(data, window=14):
    # Ensure the 'Close' column is correctly accessed, even if there's a multi-level column index
    if isinstance(data.columns, pd.MultiIndex):
        # If the DataFrame has a multi-level column index, access the 'Close' column using the level name
        close_prices = data['Close'].iloc[:, 0]  # Assuming 'Close' is the first level under 'Ticker'
    else:
        # If the DataFrame has a single-level column index, access 'Close' directly
        close_prices = data['Close']

    # Calculate the differences between consecutive closing prices
    delta = close_prices.diff()

    # Separate the positive and negative differences
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # Calculate the rolling averages of gains and losses
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()

    # Calculate the Relative Strength (RS) and the Relative Strength Index (RSI)
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

In [ ]:
def calculate_macd(data, fast=12, slow=26, signal=9):
    # Make sure we're working with a copy of the data to avoid warnings
    close_series = data['Close'].copy()

    # Calculate EMAs
    ema_fast = close_series.ewm(span=fast, adjust=False).mean()
    ema_slow = close_series.ewm(span=slow, adjust=False).mean()

    # Calculate MACD components
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    histogram = macd_line - signal_line

    return macd_line, signal_line, histogram

In [ ]:
def calculate_momentum(data, period=20):
    momentum = data['Close'] / data['Close'].shift(period) - 1
    return momentum * 100

In [ ]:
def calculate_momentum_index(data, period=126):
    returns = data['Close'].pct_change().dropna()
    momentum_std = returns.rolling(window=period).std() * np.sqrt(252)
    return momentum_std

In [ ]:
def get_fundamental_data(symbol):
    try:
        ticker = yf.Ticker(f"{symbol}.NS")
        info = ticker.info
        fundamental_data = {
            'P/E Ratio': info.get('trailingPE', 'N/A'),
            'Forward P/E': info.get('forwardPE', 'N/A'),
            'Market Cap': info.get('marketCap', 'N/A'),
            'EPS': info.get('trailingEps', 'N/A'),
            'Dividend Yield': info.get('dividendYield', 'N/A'),
            'Debt to Equity': info.get('debtToEquity', 'N/A'),
            'Return on Equity': info.get('returnOnEquity', 'N/A'),
            'Revenue Growth': info.get('revenueGrowth', 'N/A'),
            'Profit Margins': info.get('profitMargins', 'N/A'),
            'Beta': info.get('beta', 'N/A'),
            'Current Ratio': info.get('currentRatio', 'N/A'),
            'Book Value': info.get('bookValue', 'N/A'),
            '52-Week High': info.get('fiftyTwoWeekHigh', 'N/A'),
            '52-Week Low': info.get('fiftyTwoWeekLow', 'N/A'),
            'Target Price': info.get('targetMeanPrice', 'N/A')
        }
        return fundamental_data
    except Exception as e:
        print(f"Error fetching fundamental data for {symbol}: {e}")
        return {}

In [ ]:
def determine_strength(data, rsi, macd_line, signal_line):
    # Ensure data is not empty
    if isinstance(data, pd.DataFrame) and data.empty:
        return [], []

    current_price = data['Close'].iloc[-1]
    sma_50 = data['Close'].rolling(window=50).mean().iloc[-1]
    sma_200 = data['Close'].rolling(window=200).mean().iloc[-1]

    strengths = []
    weaknesses = []

    # RSI Analysis
    if not isinstance(rsi, pd.Series) or len(rsi) == 0:
        print("RSI data is empty")
    else:
        last_rsi = rsi.iloc[-1]
        if isinstance(last_rsi, pd.Series):
            last_rsi = last_rsi.item()  # Convert to scalar if it's a Series
        if last_rsi > 70:
            weaknesses.append("RSI indicates overbought conditions")
        elif last_rsi < 30:
            strengths.append("RSI indicates oversold conditions (potential buying opportunity)")
        elif 40 <= last_rsi <= 60:
            strengths.append("RSI in neutral zone showing balance between buyers and sellers")
        elif 60 < last_rsi < 70:
            strengths.append("Strong RSI showing positive momentum")

    # MACD Analysis
    if (not isinstance(macd_line, pd.Series) or len(macd_line) == 0 or
        not isinstance(signal_line, pd.Series) or len(signal_line) == 0):
        print("MACD or signal line data is empty")
    else:
        macd_value = macd_line.iloc[-1]
        signal_value = signal_line.iloc[-1]
        if isinstance(macd_value, pd.Series):
            macd_value = macd_value.item()
        if isinstance(signal_value, pd.Series):
            signal_value = signal_value.item()
        if macd_value > signal_value:
            strengths.append("MACD line above signal line indicating bullish momentum")
        else:
            weaknesses.append("MACD line below signal line indicating bearish momentum")

    # Moving Average Analysis
    if isinstance(sma_50, (int, float)) or (isinstance(sma_50, pd.Series) and pd.notna(sma_50.iloc[0])):
        if isinstance(current_price, pd.Series):
            current_price = current_price.iloc[0]  # Get scalar value
        if isinstance(sma_50, pd.Series):
            sma_50 = sma_50.iloc[0]  # Get scalar value

        if current_price > sma_50:
            strengths.append("Price above 50-day SMA showing short-term strength")
        else:
            weaknesses.append("Price below 50-day SMA showing short-term weakness")
    else:
        print("SMA 50 data is invalid")

    if isinstance(sma_200, (int, float)) or (isinstance(sma_200, pd.Series) and pd.notna(sma_200.iloc[0])):
        if isinstance(current_price, pd.Series):
            current_price = current_price.iloc[0]  # Get scalar value
        if isinstance(sma_200, pd.Series):
            sma_200 = sma_200.iloc[0]  # Get scalar value

        if current_price > sma_200:
            strengths.append("Price above 200-day SMA suggesting long-term uptrend")
        else:
            weaknesses.append("Price below 200-day SMA suggesting long-term downtrend")
    else:
        print("SMA 200 data is invalid")

    # Golden/Death Cross
    if pd.notna(sma_50) and pd.notna(sma_200):  # Use pd.notna instead of not pd.isna
        if sma_50 > sma_200 and (sma_50 / sma_200 - 1) < 0.03:
            strengths.append("Recent golden cross or nearing golden cross (50-day SMA crossing above 200-day SMA)")
        elif sma_50 < sma_200 and (sma_200 / sma_50 - 1) < 0.03:
            weaknesses.append("Recent death cross or nearing death cross (50-day SMA crossing below 200-day SMA)")
    else:
        print("SMA data is invalid for golden/death cross analysis")

    # Volume Analysis
    avg_volume = data['Volume'].mean()
    if isinstance(avg_volume, pd.Series):
        avg_volume = avg_volume.item()  # Convert to scalar if it's a Series

    recent_volume = data['Volume'].iloc[-5:].mean()
    if isinstance(recent_volume, pd.Series):
        recent_volume = recent_volume.item()  # Convert to scalar if it's a Series

    if recent_volume > avg_volume * 1.2:
        strengths.append("Above average recent trading volume showing increased interest")
    elif recent_volume < avg_volume * 0.8:
        weaknesses.append("Below average recent trading volume showing decreased interest")

    # Price Movement
    if len(data['Close']) >= 22:  # Ensure enough data for 22-day calculation
        latest_price = data['Close'].iloc[-1]
        price_22_days_ago = data['Close'].iloc[-22]

        # Convert to scalar values if they're Series
        if isinstance(latest_price, pd.Series):
            latest_price = latest_price.item()
        if isinstance(price_22_days_ago, pd.Series):
            price_22_days_ago = price_22_days_ago.item()

        monthly_return = (latest_price / price_22_days_ago - 1) * 100

        if monthly_return > 5:
            strengths.append(f"Strong monthly return of {monthly_return:.2f}%")
        elif monthly_return < -5:
            weaknesses.append(f"Weak monthly return of {monthly_return:.2f}%")
    else:
        print("Not enough data for monthly return calculation")

    # Volatility
    if len(data['Close']) >= 2:  # Ensure enough data for volatility calculation
        returns = data['Close'].pct_change().dropna()
        if len(returns) > 0:  # Make sure returns is not empty
            volatility = returns.std() * np.sqrt(252) * 100

            # Convert to scalar if it's a Series
            if isinstance(volatility, pd.Series):
                volatility = volatility.item()

            if volatility > 30:
                weaknesses.append(f"High volatility ({volatility:.2f}%) indicating increased risk")
            elif volatility < 15:
                strengths.append(f"Low volatility ({volatility:.2f}%) indicating stability")
        else:
            print("No valid returns data for volatility calculation")
    else:
        print("Not enough data for volatility calculation")

    return strengths, weaknesses

In [ ]:
# Function to create a simple profit projection
# Function to create a simple profit projection
def create_profit_projection(data, fundamentals):
    try:
        # Get historical returns
        returns = data['Close'].pct_change().dropna()
        annual_return = returns.mean() * 252 * 100

        # Use historical volatility for risk assessment
        volatility = returns.std() * np.sqrt(252) * 100

        # Current price - ensure it's a scalar value
        current_price = float(data['Close'].iloc[-1])

        # Simple projection scenarios
        conservative_growth = annual_return * 0.5
        moderate_growth = annual_return
        aggressive_growth = annual_return * 1.5

        # 1-year price targets
        conservative_target = current_price * (1 + conservative_growth/100)
        moderate_target = current_price * (1 + moderate_growth/100)
        aggressive_target = current_price * (1 + aggressive_growth/100)

        # Adjust based on fundamentals if available
        pe_ratio = fundamentals.get('P/E Ratio', None)
        if pe_ratio and pe_ratio != 'N/A':
            # Adjust projections based on P/E ratio relative to sector average
            if float(pe_ratio) > 25:
                conservative_target *= 0.9
                moderate_target *= 0.95
            elif float(pe_ratio) < 15:
                moderate_target *= 1.05
                aggressive_target *= 1.1

        return {
            'Current Price': round(float(current_price), 2),
            'Annual Return (Historical)': f"{float(annual_return):.2f}%",
            'Volatility': f"{float(volatility):.2f}%",
            'Conservative 1-Year Target': round(float(conservative_target), 2),
            'Moderate 1-Year Target': round(float(moderate_target), 2),
            'Aggressive 1-Year Target': round(float(aggressive_target), 2)
        }
    except Exception as e:
        print(f"Error creating profit projection: {e}")
        return {
            'Current Price': float(data['Close'].iloc[-1]) if not data.empty else 'N/A',
            'Annual Return (Historical)': 'N/A',
            'Volatility': 'N/A',
            'Conservative 1-Year Target': 'N/A',
            'Moderate 1-Year Target': 'N/A',
            'Aggressive 1-Year Target': 'N/A'
        }

In [ ]:
# Function to generate sentiment analysis
def generate_sentiment(data, rsi, macd_line, signal_line, fundamentals, mom_6m, mom_1y):
    # Technical signals

    # Fix: Extract scalar values for comparison
    rsi_value = rsi.iloc[-1]
    if isinstance(rsi_value, (pd.Series, pd.DataFrame)):
        rsi_value = rsi_value.iloc[0]  # Or .values[0] depending on your data structure

    rsi_signal = 1 if rsi_value < 70 and rsi_value > 40 else -1

    # Same fix for MACD
    macd_value = macd_line.iloc[-1]
    signal_value = signal_line.iloc[-1]
    if isinstance(macd_value, (pd.Series, pd.DataFrame)):
        macd_value = macd_value.iloc[0]
    if isinstance(signal_value, (pd.Series, pd.DataFrame)):
        signal_value = signal_value.iloc[0]

    macd_signal = 1 if macd_value > signal_value else -1

    # Price trends
    close_value = data['Close'].iloc[-1]
    ma_50_value = data['Close'].rolling(window=50).mean().iloc[-1]
    ma_200_value = data['Close'].rolling(window=200).mean().iloc[-1]

    # Ensure we have scalar values
    if isinstance(close_value, (pd.Series, pd.DataFrame)):
        close_value = close_value.item()  # .item() is another way to extract a scalar value
    if isinstance(ma_50_value, (pd.Series, pd.DataFrame)):
        ma_50_value = ma_50_value.item()
    if isinstance(ma_200_value, (pd.Series, pd.DataFrame)):
        ma_200_value = ma_200_value.item()

    price_above_50ma = 1 if close_value > ma_50_value else -1
    price_above_200ma = 1 if close_value > ma_200_value else -1

    # Momentum - Fix: Extract scalar value for comparison
    mom_value = mom_6m.iloc[-1]
    if isinstance(mom_value, (pd.Series, pd.DataFrame)):
        mom_value = mom_value.iloc[0]  # Or .values[0] depending on structure
    momentum_signal = 1 if mom_value < 0.15 else (0 if mom_value < 0.25 else -1)

    # Volume - Fix: Extract scalar values for comparison
    recent_vol_mean = data['Volume'].iloc[-10:].mean()
    older_vol_mean = data['Volume'].iloc[-30:].mean()
    # Ensure we have scalar values
    if isinstance(recent_vol_mean, (pd.Series, pd.DataFrame)):
        recent_vol_mean = recent_vol_mean.iloc[0]
    if isinstance(older_vol_mean, (pd.Series, pd.DataFrame)):
        older_vol_mean = older_vol_mean.iloc[0]
    volume_trend = 1 if recent_vol_mean > older_vol_mean else -1

    # Fundamentals
    pe_signal = 0
    if fundamentals.get('P/E Ratio', 'N/A') != 'N/A':
        pe = float(fundamentals['P/E Ratio'])
        pe_signal = 1 if pe < 20 else (0 if pe < 30 else -1)

    # Combine signals
    signals = [rsi_signal, macd_signal, price_above_50ma, price_above_200ma, momentum_signal, volume_trend, pe_signal]
    valid_signals = [s for s in signals if s != 0]

    if not valid_signals:
        sentiment = "Neutral"
        recommendation = "Hold"
        score = 0
    else:
        score = sum(valid_signals) / len(valid_signals)

        if score > 0.5:
            sentiment = "Strongly Positive"
            recommendation = "Strong Buy"
        elif score > 0:
            sentiment = "Positive"
            recommendation = "Buy"
        elif score > -0.5:
            sentiment = "Negative"
            recommendation = "Sell"
        else:
            sentiment = "Strongly Negative"
            recommendation = "Strong Sell"

    return {
        'Sentiment': sentiment,
        'Recommendation': recommendation,
        'Score': f"{score:.2f}",
        'RSI Signal': "Positive" if rsi_signal > 0 else "Negative",
        'MACD Signal': "Positive" if macd_signal > 0 else "Negative",
        'Price Trend': "Positive" if price_above_50ma > 0 else "Negative",
        'Long-term Trend': "Positive" if price_above_200ma > 0 else "Negative",
        'Momentum Signal': "Positive" if momentum_signal > 0 else ("Neutral" if momentum_signal == 0 else "Negative"),
        'Volume Trend': "Positive" if volume_trend > 0 else "Negative"
    }

In [ ]:
# Function to generate markdown content for a stock
def generate_stock_markdown(company_name, symbol, data, fundamentals, strengths, weaknesses, profit_projection, mom_6m, mom_1y, sentiment):
    if data.empty:
        return f"# {company_name} ({symbol})\n\nError: Unable to fetch data for this stock."

    # Format dates for readability
    # data.index = data.index.strftime('%Y-%m-%d')
    data.index = pd.to_datetime(data.index)

    # Start building markdown content
    md_content = f"# {company_name} ({symbol})\n\n"

    # Add last updated timestamp
    md_content += f"*Last Updated: {dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*\n\n"

    # Current price and summary
    current_price = data['Close'].iloc[-1]
    price_change = (current_price / data['Close'].iloc[0] - 1) * 100

    md_content += f"## Current Price and Performance\n\n"
    md_content += f"**Current Price:** ₹{float(current_price):.2f}\n\n"
    md_content += f"**1-Year Change:** {float(price_change):.2f}%\n\n"

    # Add sentiment and recommendation
    md_content += f"## Sentiment Analysis\n\n"
    md_content += f"**Sentiment:** {sentiment['Sentiment']}\n\n"
    md_content += f"**Recommendation:** {sentiment['Recommendation']}\n\n"
    md_content += f"**Sentiment Score:** {sentiment['Score']}\n\n"

    md_content += "### Sentiment Factors\n\n"
    md_content += f"- **RSI Signal:** {sentiment['RSI Signal']}\n"
    md_content += f"- **MACD Signal:** {sentiment['MACD Signal']}\n"
    md_content += f"- **Price Trend (50-day MA):** {sentiment['Price Trend']}\n"
    md_content += f"- **Long-term Trend (200-day MA):** {sentiment['Long-term Trend']}\n"
    md_content += f"- **Momentum Signal:** {sentiment['Momentum Signal']}\n"
    md_content += f"- **Volume Trend:** {sentiment['Volume Trend']}\n\n"

    # Add strengths and weaknesses
    md_content += "## Strengths\n\n"
    for strength in strengths:
        md_content += f"- {strength}\n"
    md_content += "\n"

    md_content += "## Weaknesses\n\n"
    for weakness in weaknesses:
        md_content += f"- {weakness}\n"
    md_content += "\n"

    # Add momentum index - Fixed this section
    md_content += "## Momentum Analysis\n\n"
    if not mom_6m.empty and not pd.isna(mom_6m.iloc[-1]).any():
        md_content += f"**6-Month Momentum Index:** {float(mom_6m.iloc[-1]):.4f}\n\n"
    else:
        md_content += "**6-Month Momentum Index:** N/A\n\n"

    if not mom_1y.empty and not pd.isna(mom_1y.iloc[-1]).any():
        md_content += f"**1-Year Momentum Index:** {float(mom_1y.iloc[-1]):.4f}\n\n"
    else:
        md_content += "**1-Year Momentum Index:** N/A\n\n"

    # Add technical indicators
    rsi = calculate_rsi(data)
    macd_line, signal_line, histogram = calculate_macd(data)

    md_content += "## Technical Indicators\n\n"
    md_content += f"**RSI (14-day):** {float(rsi.iloc[-1]):.2f}\n\n"
    md_content += f"**MACD Line:** {float(macd_line.iloc[-1]):.2f}\n\n"
    md_content += f"**MACD Signal Line:** {float(signal_line.iloc[-1]):.2f}\n\n"
    md_content += f"**MACD Histogram:** {float(histogram.iloc[-1]):.2f}\n\n"

    # Add moving averages
    sma_50 = data['Close'].rolling(window=50).mean().iloc[-1]
    sma_200 = data['Close'].rolling(window=200).mean().iloc[-1]
    md_content += f"**50-day SMA:** ₹{float(sma_50):.2f}\n\n"
    md_content += f"**200-day SMA:** ₹{float(sma_200):.2f}\n\n"

    # Add fundamental analysis
    md_content += "## Fundamental Analysis\n\n"

    for key, value in fundamentals.items():
        # Format percentage values for better readability
        if key in ['Dividend Yield', 'Revenue Growth', 'Profit Margins', 'Return on Equity'] and value != 'N/A':
            try:
                value = f"{float(value) * 100:.2f}%"
            except:
                pass
        md_content += f"**{key}:** {value}\n\n"

    # Add profit projections
    md_content += "## Profit Projections\n\n"

    for key, value in profit_projection.items():
        md_content += f"**{key}:** {value}\n\n"

    # Add price history table (last 12 months, monthly data)
    md_content += "## Price History (Monthly)\n\n"

    # Group by month and take last trading day of each month
    monthly_data = data.groupby(pd.Grouper(freq='M')).last().reset_index()

    md_content += "| Date | Open | High | Low | Close | Volume |\n"
    md_content += "|------|------|------|-----|-------|--------|\n"

    for index, row in monthly_data.iterrows():
        # Get the date from the row structure
        # if 'Date' in row:
        #     date_value = row['Date']
        #     if hasattr(date_value, 'values') and len(date_value.values) > 0:
        #         date_str = str(date_value.values[0])
        #     else:
        #         date_str = str(date_value)
        # else:
        #     # Fallback - use the first element if it looks like a date
        #     first_element = next(iter(row.items()))[1]
        #     if isinstance(first_element, (pd.Timestamp, datetime.datetime)):
        #         date_str = first_element.strftime('%Y-%m-%d')
        #     else:
        #         date_str = str(index)
        # Extract date from the string representation
        row_str = str(row['Date'])
        date_parts = row_str.split('\n')

        # The date should be in the second part after "Ticker"
        if len(date_parts) > 1:
            date_line = date_parts[1].strip()
            date_str = date_line
        else:
            date_str = str(index)

        md_content += f"| {date_str} | ₹{float(row['Open']):.2f} | ₹{float(row['High']):.2f} | ₹{float(row['Low']):.2f} | ₹{float(row['Close']):.2f} | {int(float(row['Volume'])):,} |\n"

    return md_content

In [ ]:
# Function to create batch markdown file for multiple stocks
def create_batch_markdown(batch_results, batch_number):
    md_content = f"# NSE Stocks Analysis - Batch {batch_number}\n\n"
    md_content += f"*Last Updated: {dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*\n\n"

    md_content += "## Stocks in this Batch\n\n"

    # Create a table of contents
    md_content += "| Company | Symbol | Current Price | 1-Year Change | Sentiment | Recommendation |\n"
    md_content += "|---------|--------|---------------|---------------|-----------|----------------|\n"

    for result in batch_results:
        company_name = result['company_name']
        symbol = result['symbol']

        if not result['data'].empty:
            current_price = result['data']['Close'].iloc[-1]
            price_change = (current_price / result['data']['Close'].iloc[0] - 1) * 100
            sentiment = result['sentiment']['Sentiment']
            recommendation = result['sentiment']['Recommendation']

            # Fix: Extract the numeric values from Series if they are Series
            current_price = float(current_price) if isinstance(current_price, (pd.Series, pd.DataFrame)) else current_price
            price_change = float(price_change) if isinstance(price_change, (pd.Series, pd.DataFrame)) else price_change

            md_content += f"| {company_name} | {symbol} | ₹{current_price:.2f} | {price_change:.2f}% | {sentiment} | {recommendation} |\n"
        else:
            md_content += f"| {company_name} | {symbol} | N/A | N/A | N/A | N/A |\n"

    md_content += "\n\n"

    # Add individual stock analyses
    for result in batch_results:
        md_content += result['markdown']
        md_content += "\n\n---\n\n"  # Add separator between stocks

    return md_content

In [ ]:
# Function to create an index file
def create_index_file(stock_list, output_dir, batch_files):
    index_md = "# NSE Stocks Analysis\n\n"
    index_md += f"*Generated on: {dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*\n\n"

    index_md += "## Available Stocks\n\n"
    index_md += "| Company | Symbol | File Link |\n"
    index_md += "|---------|--------|----------|\n"

    for company_name, symbol in stock_list:
        index_md += f"| {company_name} | {symbol} | [{symbol}.md]({symbol}.md) |\n"

    index_md += "\n\n## Batch Files\n\n"

    for i, batch_file in enumerate(batch_files):
        batch_name = os.path.basename(batch_file)
        index_md += f"- [Batch {i+1}]({batch_name})\n"

    with open(f"{output_dir}/index.md", "w", encoding='utf-8') as f:
        f.write(index_md)



In [ ]:
# Parse the stock list from the user input
def parse_stock_list(stock_list_text):
    stocks = []
    lines = stock_list_text.strip().split('\n')

    for line in lines:
        if '\t' in line:
            parts = line.split('\t')
            if len(parts) >= 2:
                company_name = parts[0].strip()
                symbol = parts[1].strip()
                if company_name not in ['Company Name', ''] and symbol not in ['Symbol', '']:
                    stocks.append((company_name, symbol))

    return stocks

In [ ]:
# Main function
def analyze_stocks(stock_list, output_dir="stock_analysis/reports", batch_size=50):
    os.makedirs(output_dir, exist_ok=True)

    # Process stocks in batches
    batch_number = 1
    batch_results = []
    batch_files = []

    for i, (company_name, symbol) in enumerate(stock_list):
        print(f"Processing {i+1}/{len(stock_list)}: {company_name} ({symbol})")

        # Get stock data
        data = get_stock_data(symbol)

        if not data.empty:
            # Calculate technical indicators
            rsi = calculate_rsi(data)
            macd_line, signal_line, histogram = calculate_macd(data)
            mom_6m = calculate_momentum_index(data, period=126)  # ~6 months
            mom_1y = calculate_momentum_index(data, period=252)  # ~1 year

            # Get fundamental data
            fundamentals = get_fundamental_data(symbol)

            # Determine strengths and weaknesses
            strengths, weaknesses = determine_strength(data, rsi, macd_line, signal_line)

            # Create profit projection
            profit_projection = create_profit_projection(data, fundamentals)

            # Generate sentiment analysis
            sentiment = generate_sentiment(data, rsi, macd_line, signal_line, fundamentals, mom_6m, mom_1y)
        else:
            fundamentals = {}
            strengths = []
            weaknesses = []
            profit_projection = {}
            mom_6m = pd.Series()
            mom_1y = pd.Series()
            sentiment = {
                'Sentiment': 'N/A',
                'Recommendation': 'N/A',
                'Score': 'N/A',
                'RSI Signal': 'N/A',
                'MACD Signal': 'N/A',
                'Price Trend': 'N/A',
                'Long-term Trend': 'N/A',
                'Momentum Signal': 'N/A',
                'Volume Trend': 'N/A'
            }

        # Generate markdown for individual stock
        stock_md = generate_stock_markdown(
            company_name, symbol, data, fundamentals, strengths, weaknesses,
            profit_projection, mom_6m, mom_1y, sentiment
        )

        # Save individual stock markdown
        with open(f"{output_dir}/{symbol}.md", "w", encoding='utf-8') as f:
            f.write(stock_md)

        # # Save file to drive
        # with open(f"/content/drive/MyDrive/Colab Notebooks/stocks_v2/Reports/{symbol}.md", 'w') as f:
        #     f.write(stock_md)
        # Add to batch results
        batch_results.append({
            'company_name': company_name,
            'symbol': symbol,
            'data': data,
            'sentiment': sentiment,
            'markdown': stock_md
        })

        # # If batch is complete or it's the last stock, create batch file
        # if (i + 1) % batch_size == 0 or i == len(stock_list) - 1:
        #     batch_md = create_batch_markdown(batch_results, batch_number)
        #     batch_file = f"{output_dir}/Batch_{batch_number}.md"

        #     with open(batch_file, "w", encoding='utf-8') as f:
        #         f.write(batch_md)

        #     batch_files.append(batch_file)
        #     batch_results = []
        #     batch_number += 1

        #     # Allow some time between batches to avoid rate limiting
        #     time.sleep(random.uniform(1, 3))

    # # Create index file
    # create_index_file(stock_list, output_dir, batch_files)

    try:
            if os.name == 'nt':  # Windows
                import shutil
                shutil.make_archive(output_dir, 'zip', output_dir)
            else:  # Linux/Mac
                os.system(f'zip -r {output_dir}.zip {output_dir}')

            print(f"\nAnalysis complete! Files saved to {output_dir}")
            print(f"Zip file created at: {output_dir}.zip")
    except Exception as e:
        print(f"Error creating zip file: {e}")
        print(f"Analysis complete! Files saved to {output_dir}")
    return batch_files


In [ ]:
# get_stock_data("RELIANCE")

In [ ]:
# stock_list_text  = """Company Name	Symbol
# Zydus Wellness Ltd.	ZYDUSWELL
# eMudhra Ltd.	EMUDHRA"""
# stocks_df = parse_stock_list(stock_list_text)
# analyze_stocks(stocks_df)

In [ ]:
stock_list_text1 = """Company Name	Symbol
Rail Vikas Nigam Ltd.	RVNL
Adani Enterprises Ltd.	ADANIENT
Adani Ports and Special Economic Zone Ltd.	ADANIPORTS
Apollo Hospitals Enterprise Ltd.	APOLLOHOSP
Asian Paints Ltd.	ASIANPAINT
Axis Bank Ltd.	AXISBANK
Bajaj Auto Ltd.	BAJAJ-AUTO
Bajaj Finance Ltd.	BAJFINANCE
Bajaj Finserv Ltd.	BAJAJFINSV
Bharat Electronics Ltd.	BEL
Bharti Airtel Ltd.	BHARTIARTL
Cipla Ltd.	CIPLA
Coal India Ltd.	COALINDIA
Dr. Reddy's Laboratories Ltd.	DRREDDY
Eicher Motors Ltd.	EICHERMOT
Grasim Industries Ltd.	GRASIM
HCL Technologies Ltd.	HCLTECH
HDFC Bank Ltd.	HDFCBANK
HDFC Life Insurance Company Ltd.	HDFCLIFE
Hero MotoCorp Ltd.	HEROMOTOCO
Hindalco Industries Ltd.	HINDALCO
Hindustan Unilever Ltd.	HINDUNILVR
ICICI Bank Ltd.	ICICIBANK
ITC Ltd.	ITC
IndusInd Bank Ltd.	INDUSINDBK
Infosys Ltd.	INFY
JSW Steel Ltd.	JSWSTEEL
Jio Financial Services Ltd.	JIOFIN
Kotak Mahindra Bank Ltd.	KOTAKBANK
Larsen & Toubro Ltd.	LT
Mahindra & Mahindra Ltd.	M&M
Maruti Suzuki India Ltd.	MARUTI
NTPC Ltd.	NTPC
Nestle India Ltd.	NESTLEIND
Oil & Natural Gas Corporation Ltd.	ONGC
Power Grid Corporation of India Ltd.	POWERGRID
Reliance Industries Ltd.	RELIANCE
SBI Life Insurance Company Ltd.	SBILIFE
Shriram Finance Ltd.	SHRIRAMFIN
State Bank of India	SBIN
Sun Pharmaceutical Industries Ltd.	SUNPHARMA
Tata Consultancy Services Ltd.	TCS
Tata Consumer Products Ltd.	TATACONSUM
Tata Motors Ltd.	TATAMOTORS
Tata Steel Ltd.	TATASTEEL
Tech Mahindra Ltd.	TECHM
Titan Company Ltd.	TITAN
Trent Ltd.	TRENT
UltraTech Cement Ltd.	ULTRACEMCO
Wipro Ltd.	WIPRO
Zomato Ltd.	ZOMATO360 ONE WAM Ltd.	360ONE
3M India Ltd.	3MINDIA
ACC Ltd.	ACC
AIA Engineering Ltd.	AIAENG
APL Apollo Tubes Ltd.	APLAPOLLO
AU Small Finance Bank Ltd.	AUBANK
Abbott India Ltd.	ABBOTINDIA
Adani Total Gas Ltd.	ATGL
Adani Wilmar Ltd.	AWL
Aditya Birla Capital Ltd.	ABCAPITAL
Aditya Birla Fashion and Retail Ltd.	ABFRL
Ajanta Pharmaceuticals Ltd.	AJANTPHARM
Alkem Laboratories Ltd.	ALKEM
Apar Industries Ltd.	APARINDS
Apollo Tyres Ltd.	APOLLOTYRE
Ashok Leyland Ltd.	ASHOKLEY
Astral Ltd.	ASTRAL
Aurobindo Pharma Ltd.	AUROPHARMA
BSE Ltd.	BSE
Balkrishna Industries Ltd.	BALKRISIND
Bandhan Bank Ltd.	BANDHANBNK
Bank of India	BANKINDIA
Bank of Maharashtra	MAHABANK
Berger Paints India Ltd.	BERGEPAINT
Bharat Dynamics Ltd.	BDL
Bharat Forge Ltd.	BHARATFORG
Bharat Heavy Electricals Ltd.	BHEL
Bharti Hexacom Ltd.	BHARTIHEXA
Biocon Ltd.	BIOCON
Blue Star Ltd.	BLUESTARCO
CRISIL Ltd.	CRISIL
Cochin Shipyard Ltd.	COCHINSHIP
Coforge Ltd.	COFORGE
Colgate Palmolive (India) Ltd.	COLPAL
Container Corporation of India Ltd.	CONCOR
Coromandel International Ltd.	COROMANDEL
Cummins India Ltd.	CUMMINSIND
Dalmia Bharat Ltd.	DALBHARAT
Deepak Nitrite Ltd.	DEEPAKNTR
Dixon Technologies (India) Ltd.	DIXON
Emami Ltd.	EMAMILTD
Endurance Technologies Ltd.	ENDURANCE
Escorts Kubota Ltd.	ESCORTS
Exide Industries Ltd.	EXIDEIND
FSN E-Commerce Ventures Ltd.	NYKAA
Federal Bank Ltd.	FEDERALBNK
Fortis Healthcare Ltd.	FORTIS
GE Vernova T&D India Ltd.	GVT&D
GMR Airports Ltd.	GMRAIRPORT
General Insurance Corporation of India	GICRE
Gland Pharma Ltd.	GLAND
Glaxosmithkline Pharmaceuticals Ltd.	GLAXO
Glenmark Pharmaceuticals Ltd.	GLENMARK
Global Health Ltd.	MEDANTA
Godrej Industries Ltd.	GODREJIND
Godrej Properties Ltd.	GODREJPROP
Gujarat Fluorochemicals Ltd.	FLUOROCHEM
Gujarat Gas Ltd.	GUJGASLTD
HDFC Asset Management Company Ltd.	HDFCAMC
Hindustan Petroleum Corporation Ltd.	HINDPETRO
Hindustan Zinc Ltd.	HINDZINC
Hitachi Energy India Ltd.	POWERINDIA
Honeywell Automation India Ltd.	HONAUT
Housing & Urban Development Corporation Ltd.	HUDCO
IDFC First Bank Ltd.	IDFCFIRSTB
IRB Infrastructure Developers Ltd.	IRB
Indian Bank	INDIANB
Indian Railway Catering And Tourism Corporation Ltd.	IRCTC
Indian Renewable Energy Development Agency Ltd.	IREDA
Indraprastha Gas Ltd.	IGL
Indus Towers Ltd.	INDUSTOWER
Ipca Laboratories Ltd.	IPCALAB
J.K. Cement Ltd.	JKCEMENT
JSW Infrastructure Ltd.	JSWINFRA
Jindal Stainless Ltd.	JSL
Jubilant Foodworks Ltd.	JUBLFOOD
K.P.R. Mill Ltd.	KPRMILL
KEI Industries Ltd.	KEI
KPIT Technologies Ltd.	KPITTECH
Kalyan Jewellers India Ltd.	KALYANKJIL
L&T Finance Ltd.	LTF
L&T Technology Services Ltd.	LTTS
LIC Housing Finance Ltd.	LICHSGFIN
Linde India Ltd.	LINDEINDIA
Lloyds Metals And Energy Ltd.	LLOYDSME
Lupin Ltd.	LUPIN
MRF Ltd.	MRF
Mahindra & Mahindra Financial Services Ltd.	M&MFIN
Mangalore Refinery & Petrochemicals Ltd.	MRPL
Mankind Pharma Ltd.	MANKIND
Marico Ltd.	MARICO
Max Financial Services Ltd.	MFSL
Max Healthcare Institute Ltd.	MAXHEALTH
Mazagoan Dock Shipbuilders Ltd.	MAZDOCK
Motherson Sumi Wiring India Ltd.	MSUMI
Motilal Oswal Financial Services Ltd.	MOTILALOFS
MphasiS Ltd.	MPHASIS
Muthoot Finance Ltd.	MUTHOOTFIN
NHPC Ltd.	NHPC
Aadhar Housing Finance Ltd.	AADHARHFC
Aarti Industries Ltd.	AARTIIND
Aavas Financiers Ltd.	AAVAS
Action Construction Equipment Ltd.	ACE
Aditya Birla Real Estate Ltd.	ABREL
Aditya Birla Sun Life AMC Ltd.	ABSLAMC
Aegis Logistics Ltd.	AEGISLOG
Affle (India) Ltd.	AFFLE
Alembic Pharmaceuticals Ltd.	APLLTD
Alkyl Amines Chemicals Ltd.	ALKYLAMINE
Alok Industries Ltd.	ALOKINDS
Amara Raja Energy & Mobility Ltd.	ARE&M
Amber Enterprises India Ltd.	AMBER
Anand Rathi Wealth Ltd.	ANANDRATHI
Anant Raj Ltd.	ANANTRAJ
Angel One Ltd.	ANGELONE
Aptus Value Housing Finance India Ltd.	APTUS
Asahi India Glass Ltd.	ASAHIINDIA
Aster DM Healthcare Ltd.	ASTERDM
AstraZenca Pharma India Ltd.	ASTRAZEN
Atul Ltd.	ATUL
Authum Investment & Infrastructure Ltd.	AIIL
BASF India Ltd.	BASF
BEML Ltd.	BEML
BLS International Services Ltd.	BLS
Balrampur Chini Mills Ltd.	BALRAMCHIN
Bata India Ltd.	BATAINDIA
Bayer Cropscience Ltd.	BAYERCROP
Bikaji Foods International Ltd.	BIKAJI
Birlasoft Ltd.	BSOFT
Blue Dart Express Ltd.	BLUEDART
Bombay Burmah Trading Corporation Ltd.	BBTC
Brigade Enterprises Ltd.	BRIGADE
C.E. Info Systems Ltd.	MAPMYINDIA
CCL Products (I) Ltd.	CCL
CESC Ltd.	CESC
Campus Activewear Ltd.	CAMPUS
Can Fin Homes Ltd.	CANFINHOME
Caplin Point Laboratories Ltd.	CAPLIPOINT
Capri Global Capital Ltd.	CGCL
Carborundum Universal Ltd.	CARBORUNIV
Castrol India Ltd.	CASTROLIND
Ceat Ltd.	CEATLTD
Central Bank of India	CENTRALBK
Central Depository Services (India) Ltd.	CDSL
Century Plyboards (India) Ltd.	CENTURYPLY
Cera Sanitaryware Ltd	CERA
Chalet Hotels Ltd.	CHALET
Chambal Fertilizers & Chemicals Ltd.	CHAMBLFERT
Chennai Petroleum Corporation Ltd.	CHENNPETRO
Cholamandalam Financial Holdings Ltd.	CHOLAHLDNG
City Union Bank Ltd.	CUB
Clean Science and Technology Ltd.	CLEAN
Computer Age Management Services Ltd.	CAMS
Concord Biotech Ltd.	CONCORDBIO
Craftsman Automation Ltd.	CRAFTSMAN
CreditAccess Grameen Ltd.	CREDITACC
Crompton Greaves Consumer Electricals Ltd.	CROMPTON
Cyient Ltd.	CYIENT
DCM Shriram Ltd.	DCMSHRIRAM
DOMS Industries Ltd.	DOMS
Data Patterns (India) Ltd.	DATAPATTNS
Deepak Fertilisers & Petrochemicals Corp. Ltd.	DEEPAKFERT
Delhivery Ltd.	DELHIVERY
Devyani International Ltd.	DEVYANI
Dr. Lal Path Labs Ltd.	LALPATHLAB
E.I.D. Parry (India) Ltd.	EIDPARRY
EIH Ltd.	EIHOTEL
Elecon Engineering Co. Ltd.	ELECON
Elgi Equipments Ltd.	ELGIEQUIP
Engineers India Ltd.	ENGINERSIN
Eris Lifesciences Ltd.	ERIS
Fertilisers and Chemicals Travancore Ltd.	FACT
Finolex Cables Ltd.	FINCABLES
Finolex Industries Ltd.	FINPIPE
Firstsource Solutions Ltd.	FSL
Five-Star Business Finance Ltd.	FIVESTAR
Garden Reach Shipbuilders & Engineers Ltd.	GRSE
Gillette India Ltd.	GILLETTE
Go Digit General Insurance Ltd.	GODIGIT
Godawari Power & Ispat Ltd.	GPIL
Godfrey Phillips India Ltd.	GODFRYPHLP
Godrej Agrovet Ltd.	GODREJAGRO
Granules India Ltd.	GRANULES
Graphite India Ltd.	GRAPHITE
Gravita India Ltd.	GRAVITA
Great Eastern Shipping Co. Ltd.	GESHIP
Gujarat Mineral Development Corporation Ltd.	GMDCLTD
Gujarat Narmada Valley Fertilizers and Chemicals Ltd.	GNFC
Gujarat Pipavav Port Ltd.	GPPL
Gujarat State Petronet Ltd.	GSPL
H.E.G. Ltd.	HEG
HFCL Ltd.	HFCL
Happiest Minds Technologies Ltd.	HAPPSTMNDS
Himadri Speciality Chemical Ltd.	HSCL
Hindustan Copper Ltd.	HINDCOPPER
Home First Finance Company India Ltd.	HOMEFIRST
Honasa Consumer Ltd.	HONASA
IDBI Bank Ltd.	IDBI
IFCI Ltd.	IFCI
IIFL Finance Ltd.	IIFL
INOX India Ltd.	INOXINDIA
IRCON International Ltd.	IRCON
ITI Ltd.	ITI
Indegene Ltd.	INDGN
India Cements Ltd.	INDIACEM
Indiamart Intermesh Ltd.	INDIAMART
Indian Energy Exchange Ltd.	IEX
Indian Overseas Bank	IOB
Inox Wind Ltd.	INOXWIND
Intellect Design Arena Ltd.	INTELLECT
J.B. Chemicals & Pharmaceuticals Ltd.	JBCHEPHARM
JBM Auto Ltd.	JBMA
JK Tyre & Industries Ltd.	JKTYRE
JM Financial Ltd.	JMFINANCIL
JSW Holdings Ltd.	JSWHL
Jaiprakash Power Ventures Ltd.	JPPOWER
Jammu & Kashmir Bank Ltd.	J&KBANK
Jindal Saw Ltd.	JINDALSAW
Jubilant Ingrevia Ltd.	JUBLINGREA
Jubilant Pharmova Ltd.	JUBLPHARMA
Jupiter Wagons Ltd.	JWL
Justdial Ltd.	JUSTDIAL
Jyothy Labs Ltd.	JYOTHYLAB
Jyoti CNC Automation Ltd.	JYOTICNC
KNR Constructions Ltd.	KNRCON
Kajaria Ceramics Ltd.	KAJARIACER
Kalpataru Projects International Ltd.	KPIL
Kansai Nerolac Paints Ltd.	KANSAINER
Karur Vysya Bank Ltd.	KARURVYSYA
Kaynes Technology India Ltd.	KAYNES
Kec International Ltd.	KEC
Kfin Technologies Ltd.	KFINTECH
Kirloskar Brothers Ltd.	KIRLOSBROS
Kirloskar Oil Eng Ltd.	KIRLOSENG
Krishna Institute of Medical Sciences Ltd.	KIMS
LT Foods Ltd.	LTFOODS
Latent View Analytics Ltd.	LATENTVIEW
Laurus Labs Ltd.	LAURUSLABS
Lemon Tree Hotels Ltd.	LEMONTREE
MMTC Ltd.	MMTC
Mahanagar Gas Ltd.	MGL
Maharashtra Seamless Ltd.	MAHSEAMLES
Manappuram Finance Ltd.	MANAPPURAM
Mastek Ltd.	MASTEK
Metropolis Healthcare Ltd.	METROPOLIS
Minda Corporation Ltd.	MINDACORP
Multi Commodity Exchange of India Ltd.	MCX
NATCO Pharma Ltd.	NATCOPHARM
NBCC (India) Ltd.	NBCC
NCC Ltd.	NCC
NMDC Steel Ltd.	NSLNISP
Narayana Hrudayalaya Ltd.	NH
Nava Ltd.	NAVA
Navin Fluorine International Ltd.	NAVINFLUOR
Netweb Technologies India Ltd.	NETWEB
Network18 Media & Investments Ltd.	NETWORK18
Neuland Laboratories Ltd.	NEULANDLAB
Newgen Software Technologies Ltd.	NEWGEN
Nuvama Wealth Management Ltd.	NUVAMA
Olectra Greentech Ltd.	OLECTRA
PCBL Chemical Ltd.	PCBL
PG Electroplast Ltd.	PGEL
PNB Housing Finance Ltd.	PNBHOUSING
PNC Infratech Ltd.	PNCINFRA
PTC Industries Ltd.	PTCIL
PVR INOX Ltd.	PVRINOX
Pfizer Ltd.	PFIZER
Piramal Enterprises Ltd.	PEL
Piramal Pharma Ltd.	PPLPHARMA
Poly Medicure Ltd.	POLYMED
Poonawalla Fincorp Ltd.	POONAWALLA
Praj Industries Ltd.	PRAJIND
Quess Corp Ltd.	QUESS
R R Kabel Ltd.	RRKABEL
RBL Bank Ltd.	RBLBANK
RHI MAGNESITA INDIA LTD.	RHIM
RITES Ltd.	RITES
Radico Khaitan Ltd	RADICO
Railtel Corporation Of India Ltd.	RAILTEL
Rainbow Childrens Medicare Ltd.	RAINBOW
Ramkrishna Forgings Ltd.	RKFORGE
Rashtriya Chemicals & Fertilizers Ltd.	RCF
RattanIndia Enterprises Ltd.	RTNINDIA
Raymond Ltd.	RAYMOND
Redington Ltd.	REDINGTON
Reliance Power Ltd.	RPOWER
Route Mobile Ltd.	ROUTE
SBFC Finance Ltd.	SBFC
SKF India Ltd.	SKFINDIA
Sammaan Capital Ltd.	SAMMAANCAP
Sapphire Foods India Ltd.	SAPPHIRE
Sarda Energy and Minerals Ltd.	SARDAEN
Saregama India Ltd	SAREGAMA
Schneider Electric Infrastructure Ltd.	SCHNEIDER
Shipping Corporation of India Ltd.	SCI
Shree Renuka Sugars Ltd.	RENUKA
Shyam Metalics and Energy Ltd.	SHYAMMETL
Signatureglobal (India) Ltd.	SIGNATURE
Sobha Ltd.	SOBHA
Sonata Software Ltd.	SONATSOFTW
Sterling and Wilson Renewable Energy Ltd.	SWSOLAR
Sumitomo Chemical India Ltd.	SUMICHEM
Suven Pharmaceuticals Ltd.	SUVENPHAR
Swan Energy Ltd.	SWANENERGY
Syrma SGS Technology Ltd.	SYRMA
TBO Tek Ltd.	TBOTEK
Tanla Platforms Ltd.	TANLA
Tata Chemicals Ltd.	TATACHEM
Tata Teleservices (Maharashtra) Ltd.	TTML
Techno Electric & Engineering Company Ltd.	TECHNOE
Tejas Networks Ltd.	TEJASNET
The Ramco Cements Ltd.	RAMCOCEM
Timken India Ltd.	TIMKEN
Titagarh Rail Systems Ltd.	TITAGARH
Transformers And Rectifiers (India) Ltd.	TARIL
Trident Ltd.	TRIDENT
Triveni Engineering & Industries Ltd.	TRIVENI
Triveni Turbine Ltd.	TRITURBINE
UCO Bank	UCOBANK
UTI Asset Management Company Ltd.	UTIAMC
Usha Martin Ltd.	USHAMART
V-Guard Industries Ltd.	VGUARD
Valor Estate Ltd.	DBREALTY
Vardhman Textiles Ltd.	VTL
Vedant Fashions Ltd.	MANYAVAR
Vijaya Diagnostic Centre Ltd.	VIJAYA
Welspun Corp Ltd.	WELCORP
Welspun Living Ltd.	WELSPUNLIV
Westlife Foodworld Ltd.	WESTLIFE
Whirlpool of India Ltd.	WHIRLPOOL
Wockhardt Ltd.	WOCKPHARMA
ZF Commercial Vehicle Control Systems India Ltd.	ZFCVINDIA
Zee Entertainment Enterprises Ltd.	ZEEL
Zen Technologies Ltd.	ZENTEC
Zensar Technolgies Ltd.	ZENSARTECH
eClerx Services Ltd.	ECLERX
AGI Greenpac Ltd.	AGI
ASK Automotive Ltd.	ASKAUTOLTD
Aarti Drugs Ltd.	AARTIDRUGS
Aarti Pharmalabs Ltd.	AARTIPHARM
Advanced Enzyme Tech Ltd.	ADVENZYMES
Aether Industries Ltd.	AETHER
Ahluwalia Contracts (India) Ltd.	AHLUCONT
Akzo Nobel India Ltd.	AKZOINDIA
Allcargo Logistics Ltd.	ALLCARGO
Ami Organics Ltd.	AMIORG
Apeejay Surrendra Park Hotels Ltd.	PARKHOTELS
Archean Chemical Industries Ltd.	ACI
Arvind Fashions Ltd.	ARVINDFASN
Arvind Ltd.	ARVIND
Ashoka Buildcon Ltd.	ASHOKA
Astra Microwave Products Ltd.	ASTRAMICRO
Aurionpro Solution Ltd.	AURIONPRO
Avalon Technologies Ltd.	AVALON
Avanti Feeds Ltd.	AVANTIFEED
Awfis Space Solutions Ltd.	AWFIS
Azad Engineering Ltd.	AZAD
Bajaj Hindusthan Sugar Ltd.	BAJAJHIND
Balaji Amines Ltd.	BALAMINES
Balu Forge Industries Ltd.	BALUFORGE
Banco Products (India) Ltd.	BANCOINDIA
Bhansali Engineering Polymers Ltd.	BEPL
Bharat Bijlee Ltd.	BBL
Birla Corporation Ltd.	BIRLACORPN
Blue Jet Healthcare Ltd.	BLUEJET
Bombay Dyeing & Manufacturing Co. Ltd.	BOMDYEING
Borosil Ltd.	BOROLTD
Borosil Renewables Ltd.	BORORENEW
CIE Automotive India Ltd.	CIEINDIA
CMS Info Systems Ltd.	CMSINFO
CSB Bank Ltd.	CSBBANK
Cartrade Tech Ltd.	CARTRADE"""

stocks_df1 = parse_stock_list(stock_list_text1)
print('len(stocks_df1)', len(stocks_df1))

In [ ]:
stock_list_text2  = """Company Name	Symbol
Cello World Ltd.	CELLO
Chemplast Sanmar Ltd.	CHEMPLASTS
Choice International Ltd.	CHOICEIN
Cigniti Technologies Ltd.	CIGNITITEC
Cyient DLM Ltd.	CYIENTDLM
DCB Bank Ltd.	DCBBANK
DCX Systems Ltd.	DCXINDIA
Datamatics Global Services Ltd.	DATAMATICS
Dhani Services Ltd.	DHANI
Dilip Buildcon Ltd.	DBL
Dishman Carbogen Amcis Ltd.	DCAL
Dodla Dairy Ltd.	DODLA
Dynamatic Technologies Ltd.	DYNAMATECH
EPL Ltd.	EPL
Easy Trip Planners Ltd.	EASEMYTRIP
Edelweiss Financial Services Ltd.	EDELWEISS
Electronics Mart India Ltd.	EMIL
Electrosteel Castings Ltd.	ELECTCAST
Entero Healthcare Solutions Ltd.	ENTERO
Epigral Ltd.	EPIGRAL
Equitas Small Finance Bank Ltd.	EQUITASBNK
Ethos Ltd.	ETHOSLTD
FDC Ltd.	FDC
Fiem Industries Ltd	FIEMIND
Fine Organic Industries Ltd.	FINEORG
Fineotex Chemical Ltd.	FCL
Force Motors Ltd.	FORCEMOT
G R Infraprojects Ltd.	GRINFRA
GHCL Ltd.	GHCL
GMM Pfaudler Ltd.	GMMPFAUDLR
GMR Power and Urban Infra Ltd.	GMRP&UI
Gabriel India Ltd.	GABRIEL
Ganesh Housing Corporation Ltd.	GANESHHOUC
Ganesha Ecosphere Ltd.	GANECOS
Garware Hi-Tech Films Ltd.	GRWRHITECH
Garware Technical Fibres Ltd.	GARFIBRES
Gateway Distriparks Ltd.	GATEWAY
Gokaldas Exports Ltd.	GOKEX
Gopal Snacks Ltd.	GOPAL
Greaves Cotton Ltd.	GREAVESCOT
Greenpanel Industries Ltd.	GREENPANEL
Greenply Industries Ltd.	GREENPLY
Gujarat Ambuja Exports Ltd.	GAEL
Gujarat State Fertilizers & Chemicals Ltd.	GSFC
Gulf Oil Lubricants India Ltd.	GULFOILLUB
H.G. Infra Engineering Ltd.	HGINFRA
Hathway Cable & Datacom Ltd.	HATHWAY
Healthcare Global Enterprises Ltd.	HCG
HeidelbergCement India Ltd.	HEIDELBERG
Hemisphere Properties India Ltd.	HEMIPROP
Heritage Foods Ltd.	HERITGFOOD
Hikal Ltd.	HIKAL
Hindustan Construction Co. Ltd.	HCC
IFB Industries Ltd.	IFBIND
ITD Cementation India Ltd.	ITDCEM
Imagicaaworld Entertainment Ltd.	IMAGICAA
India Glycols Ltd.	INDIAGLYCO
India Shelter Finance Corporation Ltd.	INDIASHLTR
Indian Metals & Ferro Alloys Ltd.	IMFA
Indigo Paints Ltd.	INDIGOPNTS
Indo Count Industries Ltd.	ICIL
Infibeam Avenues Ltd.	INFIBEAM
Ingersoll Rand (India) Ltd.	INGERRAND
Innova Captab Ltd.	INNOVACAP
Inox Green Energy Services Ltd.	INOXGREEN
Ion Exchange (India) Ltd.	IONEXCHANG
Isgec Heavy Engineering Ltd.	ISGEC
J.Kumar Infraprojects Ltd.	JKIL
JK Lakshmi Cement Ltd.	JKLAKSHMI
JK Paper Ltd.	JKPAPER
JTL Industries Ltd.	JTLIND
Jai Balaji Industries Ltd.	JAIBALAJI
Jai Corp Ltd.	JAICORPLTD
Jain Irrigation Systems Ltd.	JISLJALEQS
Jamna Auto Industries Ltd.	JAMNAAUTO
Jana Small Finance Bank Ltd.	JSFB
Jindal Worldwide Ltd.	JINDWORLD
Johnson Controls - Hitachi Air Conditioning India Ltd.	JCHAC
KPI Green Energy Ltd.	KPIGREEN
KRBL Ltd.	KRBL
KSB Ltd.	KSB
Kalyani Steels Ltd.	KSL
Karnataka Bank Ltd.	KTKBANK
Kaveri Seed Company Ltd.	KSCL
Kirloskar Pneumatic Company Ltd.	KIRLPNU
LMW Ltd.	LMW
Laxmi Organic Industries Ltd.	LXCHEM
Le Travenues Technology Ltd.	IXIGO
Lloyds Engineering Works Ltd.	LLOYDSENGG
Lux Industries Ltd.	LUXIND
MOIL Ltd.	MOIL
MSTC Ltd.	MSTCLTD
MTAR Technologies Ltd.	MTARTECH
Maharashtra Scooters Ltd.	MAHSCOOTER
Mahindra Lifespace Developers Ltd.	MAHLIFE
Man Infraconstruction Ltd.	MANINFRA
Marksans Pharma Ltd.	MARKSANS
Max Estates Ltd.	MAXESTATES
Medplus Health Services Ltd.	MEDPLUS
Mishra Dhatu Nigam Ltd.	MIDHANI
Mrs. Bectors Food Specialities Ltd.	BECTORFOOD
NEOGEN CHEMICALS LTD.	NEOGEN
NESCO Ltd.	NESCO
NOCIL Ltd.	NOCIL
National Fertilizers Ltd.	NFL
Nazara Technologies Ltd.	NAZARA
Nuvoco Vistas Corporation Ltd.	NUVOCO
Optiemus Infracom Ltd.	OPTIEMUS
Orchid Pharma Ltd.	ORCHPHARMA
Orient Cement Ltd.	ORIENTCEM
Orissa Min Dev Co Ltd.	ORISSAMINE
PC Jeweller Ltd.	PCJEWELLER
PTC India Ltd.	PTC
Paisalo Digital Ltd.	PAISALO
Paradeep Phosphates Ltd.	PARADEEP
Paras Defence and Space Technologies Ltd.	PARAS
Patel Engineering Ltd.	PATELENG
Pearl Global Industries Ltd.	PGIL
Polyplex Corporation Ltd.	POLYPLEX
Power Mech Projects Ltd.	POWERMECH
Pricol Ltd.	PRICOLLTD
Prince Pipes and Fittings Ltd.	PRINCEPIPE
Prism Johnson Ltd.	PRSMJOHNSN
Prudent Corporate Advisory Services Ltd.	PRUDENT
Rain Industries Ltd	RAIN
Rajesh Exports Ltd.	RAJESHEXPO
Rallis India Ltd.	RALLIS
Rategain Travel Technologies Ltd.	RATEGAIN
RattanIndia Power Ltd.	RTNPOWER
Redtape Ltd.	REDTAPE
Refex Industries Ltd.	REFEX
Reliance Infrastructure Ltd.	RELINFRA
Religare Enterprises Ltd.	RELIGARE
Responsive Industries Ltd.	RESPONIND
Restaurant Brands Asia Ltd.	RBA
Rossari Biotech Ltd.	ROSSARI
Safari Industries (India) Ltd.	SAFARI
Samhi Hotels Ltd.	SAMHI
Sanofi India Ltd.	SANOFI
Sansera Engineering Ltd.	SANSERA
Senco Gold Ltd.	SENCO
Sequent Scientific Ltd.	SEQUENT
Shaily Engineering Plastics Ltd.	SHAILY
Shakti Pumps (India) Ltd.	SHAKTIPUMP
Sharda Cropchem Ltd.	SHARDACROP
Share India Securities Ltd.	SHAREINDIA
Sheela Foam Ltd.	SFL
Shilpa Medicare Ltd.	SHILPAMED
Shivalik Bimetal Controls Ltd.	SBCL
Shoppers Stop Ltd.	SHOPERSTOP
Shriram Pistons & Rings Ltd.	SHRIPISTON
Skipper Ltd.	SKIPPER
South Indian Bank Ltd.	SOUTHBANK
Spandana Sphoorty Financial Ltd.	SPANDANA
Star Cement Ltd.	STARCEMENT
Sterlite Technologies Ltd.	STLTECH
Strides Pharma Science Ltd.	STAR
Stylam Industries Ltd.	STYLAMIND
Subros Ltd.	SUBROS
Sudarshan Chemical Industries Ltd.	SUDARSCHEM
Sula Vineyards Ltd.	SULA
Sun Pharma Advanced Research Company Ltd.	SPARC
Sunflag Iron & Steel Company Ltd.	SUNFLAG
Sunteck Realty Ltd.	SUNTECK
Suprajit Engineering Ltd.	SUPRAJIT
Supriya Lifescience Ltd.	SUPRIYA
Surya Roshni Ltd.	SURYAROSNI
Symphony Ltd.	SYMPHONY
TARC Ltd.	TARC
TD Power Systems Ltd.	TDPOWERSYS
TVS Supply Chain Solutions Ltd.	TVSSCS
Teamlease Services Ltd.	TEAMLEASE
Technocraft Industries (India) Ltd.	TIIL
Tega Industries Ltd.	TEGA
Texmaco Rail & Eng. Ltd.	TEXRAIL
Thangamayil Jewellery Ltd.	THANGAMAYL
The Anup Engineering Ltd.	ANUP
Thirumalai Chemicals Ltd.	TIRUMALCHM
Thomas Cook (India) Ltd.	THOMASCOOK
Tilaknagar Industries Ltd.	TI
Time Technoplast Ltd.	TIMETECHNO
Tips Music Ltd.	TIPSMUSIC
Ujjivan Small Finance Bank Ltd.	UJJIVANSFB
V-Mart Retail Ltd.	VMART
V.I.P. Industries Ltd.	VIPIND
VST Industries Ltd.	VSTIND
Va Tech Wabag Ltd.	WABAG
Vaibhav Global Ltd.	VAIBHAVGBL
Varroc Engineering Ltd.	VARROC
Venus Pipes & Tubes Ltd.	VENUSPIPES
Vesuvius India Ltd.	VESUVIUS
Voltamp Transformers Ltd	VOLTAMP
Websol Energy System Ltd.	WEBELSOLAR
Welspun Enterprises Ltd.	WELENT
Wonderla Holidays Ltd.	WONDERLA
Yatharth Hospital & Trauma Care Services Ltd.	YATHARTH
Zaggle Prepaid Ocean Services Ltd.	ZAGGLE
Zydus Wellness Ltd.	ZYDUSWELL
eMudhra Ltd.	EMUDHRA"""

stocks_df2 = parse_stock_list(stock_list_text2)
print('len(stocks_df2)', len(stocks_df2))

In [ ]:
# analyze_stocks(stocks_df1)

In [ ]:
analyze_stocks(stocks_df2)